Non-overlapping- 8x8


In [1]:
import cv2
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from skimage.util import view_as_windows
from scipy.fftpack import dct
from sklearn.preprocessing import MinMaxScaler

In [2]:
def get_patches(image_mat):
    """
    Extract patches rom an image
    :param image_mat: The image as a matrix
    :param stride: The stride of the patch extraction process
    :returns: The patches
    """
    stride=16 #stride is same as window's breadth so that it gives non-overlapping blocks.
    window_shape = (16, 16)
    image_mat=np.array(image_mat)
    
    windows = view_as_windows(image_mat, window_shape, step=stride)
#     print('windows shape:',windows.shape)

    patches = []
    for m in range(windows.shape[0]):
        for n in range(windows.shape[1]):
#             print("window shape: ",windows[m][n].shape)
            patches += [windows[m][n]]
    return patches
    

In [3]:
def std_and_ones(type_of_sub_image_blocks):
    ac_dct_stack=[]
    number_of_ones=[]

    for block in type_of_sub_image_blocks:
        dct_block = dct(block, type=2, n=None, axis=-1, norm=None, overwrite_x=False)
        dct_block_row = dct_block.flatten() # 2d dct array to 1d row array.
        ac_dct = dct_block_row[1:] # only AC component, removing the first DC comp.
        ac_dct_stack.append(ac_dct)

    ac_dct_stack=np.asarray(ac_dct_stack) #1536X63
    ac_dct_stack=ac_dct_stack.T # 63X1536

    #print("AC stacked shape: ", ac_dct_stack.shape)

    ac_dct_std = np.std(ac_dct_stack, axis=1) # row wise standard-deviation.

    for i in range(ac_dct_stack.shape[0]):
        count_one=0
        for j in range(ac_dct_stack.shape[1]):
            if(ac_dct_stack[i][j]>0):   # row wise counting number of ones.
                count_one+=1
        number_of_ones.append(count_one)

    number_of_ones=np.asarray(number_of_ones)
    
    return(ac_dct_std, number_of_ones)

In [4]:
def feature_sub_image(sub_image):
    sub_image_blocks = get_patches(sub_image) #Gives the 8x8 patches/blocks of sub_image.

    sub_image_cropped = sub_image[4:,4:] #removing 4 rows and 4 cols.
    sub_image_cropped_blocks = get_patches(sub_image_cropped)

    STD_full_image, ONE_full_image = std_and_ones(sub_image_blocks)
    STD_cropped_image, ONE_cropped_image = std_and_ones(sub_image_cropped_blocks)

    #             print("STD_full image shape: ",STD_full_image.shape)
    #             print("one_full image shape: ",ONE_full_image.shape)
    #             print("STD_crop image shape: ",STD_cropped_image.shape)
    #             print("One_crop image shape: ",ONE_cropped_image.shape)
    
    #63x4 stacked F-sub-image
    F_sub_image=np.column_stack((STD_full_image, ONE_full_image, STD_cropped_image, ONE_cropped_image))
    
    F_sub_image_flat=F_sub_image.T.flatten() #column wise flattening, 63*4=252 features
    return(F_sub_image_flat)

In [5]:
def feature_extraction(path_to_folder, class_label):
    data_list=[]
    for file_name in os.listdir(path_to_folder):
        path_to_img = os.path.join(path_to_folder,file_name)
        img = cv2.imread(path_to_img)
        
        if np.shape(img) == ():
            continue
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb) #changing to YCrCb color space.
        img_y = img[:,:,0] # the Y channel only.
        img_cr = img[:,:,1] # the Cr channel only.
        img_cb = img[:,:,2] # the Cb channel only.
        
        Fy = feature_sub_image(img_y)
        Fcr = feature_sub_image(img_cr)
        Fcb = feature_sub_image(img_cb)
#         print("fy shape: ",Fy.shape)
#         print("fcr shape: ",Fcr.shape)
#         print("fcb shape: ",Fcb.shape)
        
        final_feature = np.concatenate((Fy, Fcb, Fcr), axis=None) #63*8*3=756 flattened features.
#         print("final feature shape: ",final_feature.shape)
        
        final_feature=list(final_feature)
        final_feature.insert(0,file_name)
        final_feature.insert(1,class_label)
        data_list.append(final_feature)
        
        
    return(data_list)

In [6]:
au_path="/content/drive/MyDrive/CASIA1/CASIA_DSET/Au"
sp_path="/content/drive/MyDrive/CASIA1/Sps"
cm_path="/content/drive/MyDrive/CASIA1/Spcm"

output_name='CASIA1_features.csv'

data_list1 = feature_extraction(au_path, 0)
data_list2 = feature_extraction(sp_path, 1)
data_list3 = feature_extraction(cm_path, 2)



df = pd.DataFrame(data_list1)
df = df.append(pd.DataFrame(data_list2), ignore_index=True)
df = df.append(pd.DataFrame(data_list3), ignore_index=True)

#  --------------------------------------------------------------------------------------

# Keep this as it is
df.rename(columns = {0: "image_names", 1: "label"}, inplace = True)

scaler_norm = MinMaxScaler() 
df.iloc[:,2:] = scaler_norm.fit_transform(df.iloc[:,2:].to_numpy()) # Normalising the values in dataframe.


df.to_csv(output_name, index=False)


<ipython-input-6-636da08add45>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(data_list2), ignore_index=True)
<ipython-input-6-636da08add45>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(data_list3), ignore_index=True)


In [7]:
df.head()

,image_names,label,2,3,4,5,6,7,8,9,...,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061
0,Au_ani_0008.jpg,0,0.205975,0.199469,0.206309,0.161795,0.146440,0.132704,0.104429,0.112261,...,0.280193,0.329577,0.247012,0.335749,0.292929,0.335749,0.308605,0.410326,0.291566,0.430199
1,Au_ani_0017.jpg,0,0.069673,0.045317,0.046352,0.034725,0.039482,0.032912,0.048517,0.031099,...,0.188406,0.205634,0.071713,0.234300,0.148990,0.258454,0.118694,0.154891,0.214458,0.213675
2,Au_ani_0027.jpg,0,0.151119,0.152032,0.168488,0.166953,0.151464,0.155963,0.133673,0.153815,...,0.256039,0.298592,0.223108,0.309179,0.285354,0.292271,0.296736,0.342391,0.303614,0.384615
3,Au_ani_0010.jpg,0,0.098561,0.097059,0.079727,0.075014,0.099778,0.073798,0.054961,0.105083,...,0.200483,0.191549,0.055777,0.287440,0.189394,0.263285,0.181009,0.206522,0.226506,0.222222
4,Au_ani_0003.jpg,0,0.251978,0.257088,0.276806,0.231483,0.208593,0.261943,0.195638,0.210312,...,0.345411,0.332394,0.358566,0.381643,0.338384,0.417874,0.409496,0.323370,0.395181,0.547009


In [8]:

gf=df.drop(['label'],axis=1)

In [9]:
df.set_index('image_names', inplace=True)
gf.set_index('image_names', inplace=True)

In [10]:
df

,label,2,3,4,5,6,7,8,9,10,...,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061
image_names,,,,,,,,,,,,,,,,,,,,,
Au_ani_0008.jpg,0,0.205975,0.199469,0.206309,0.161795,0.146440,0.132704,0.104429,0.112261,0.093395,...,0.280193,0.329577,0.247012,0.335749,0.292929,0.335749,0.308605,0.410326,0.291566,0.430199
Au_ani_0017.jpg,0,0.069673,0.045317,0.046352,0.034725,0.039482,0.032912,0.048517,0.031099,0.026571,...,0.188406,0.205634,0.071713,0.234300,0.148990,0.258454,0.118694,0.154891,0.214458,0.213675
Au_ani_0027.jpg,0,0.151119,0.152032,0.168488,0.166953,0.151464,0.155963,0.133673,0.153815,0.143197,...,0.256039,0.298592,0.223108,0.309179,0.285354,0.292271,0.296736,0.342391,0.303614,0.384615
Au_ani_0010.jpg,0,0.098561,0.097059,0.079727,0.075014,0.099778,0.073798,0.054961,0.105083,0.049081,...,0.200483,0.191549,0.055777,0.287440,0.189394,0.263285,0.181009,0.206522,0.226506,0.222222
Au_ani_0003.jpg,0,0.251978,0.257088,0.276806,0.231483,0.208593,0.261943,0.195638,0.210312,0.204525,...,0.345411,0.332394,0.358566,0.381643,0.338384,0.417874,0.409496,0.323370,0.395181,0.547009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sp_S_NRN_A_nat0051_nat0051_0250.jpg,2,0.080467,0.079427,0.074821,0.037376,0.053588,0.034801,0.028702,0.043111,0.023972,...,0.159420,0.022535,0.079681,0.128019,0.103535,0.123188,0.083086,0.084239,0.110843,0.099715
Sp_S_NRN_R_arc0058_arc0058_0255.jpg,2,0.190511,0.246835,0.231900,0.232136,0.246263,0.215243,0.168278,0.221721,0.153769,...,0.243961,0.188732,0.338645,0.294686,0.202020,0.270531,0.281899,0.220109,0.293976,0.324786
Sp_S_NRN_A_nat0098_nat0098_0003.jpg,2,0.053233,0.058722,0.065177,0.061159,0.056360,0.079736,0.062008,0.057293,0.062412,...,0.265700,0.177465,0.191235,0.277778,0.181818,0.287440,0.207715,0.252717,0.224096,0.339031


In [11]:
gf

,2,3,4,5,6,7,8,9,10,11,...,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061
image_names,,,,,,,,,,,,,,,,,,,,,
Au_ani_0008.jpg,0.205975,0.199469,0.206309,0.161795,0.146440,0.132704,0.104429,0.112261,0.093395,0.079474,...,0.280193,0.329577,0.247012,0.335749,0.292929,0.335749,0.308605,0.410326,0.291566,0.430199
Au_ani_0017.jpg,0.069673,0.045317,0.046352,0.034725,0.039482,0.032912,0.048517,0.031099,0.026571,0.038780,...,0.188406,0.205634,0.071713,0.234300,0.148990,0.258454,0.118694,0.154891,0.214458,0.213675
Au_ani_0027.jpg,0.151119,0.152032,0.168488,0.166953,0.151464,0.155963,0.133673,0.153815,0.143197,0.111375,...,0.256039,0.298592,0.223108,0.309179,0.285354,0.292271,0.296736,0.342391,0.303614,0.384615
Au_ani_0010.jpg,0.098561,0.097059,0.079727,0.075014,0.099778,0.073798,0.054961,0.105083,0.049081,0.079307,...,0.200483,0.191549,0.055777,0.287440,0.189394,0.263285,0.181009,0.206522,0.226506,0.222222
Au_ani_0003.jpg,0.251978,0.257088,0.276806,0.231483,0.208593,0.261943,0.195638,0.210312,0.204525,0.162688,...,0.345411,0.332394,0.358566,0.381643,0.338384,0.417874,0.409496,0.323370,0.395181,0.547009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sp_S_NRN_A_nat0051_nat0051_0250.jpg,0.080467,0.079427,0.074821,0.037376,0.053588,0.034801,0.028702,0.043111,0.023972,0.028026,...,0.159420,0.022535,0.079681,0.128019,0.103535,0.123188,0.083086,0.084239,0.110843,0.099715
Sp_S_NRN_R_arc0058_arc0058_0255.jpg,0.190511,0.246835,0.231900,0.232136,0.246263,0.215243,0.168278,0.221721,0.153769,0.121741,...,0.243961,0.188732,0.338645,0.294686,0.202020,0.270531,0.281899,0.220109,0.293976,0.324786
Sp_S_NRN_A_nat0098_nat0098_0003.jpg,0.053233,0.058722,0.065177,0.061159,0.056360,0.079736,0.062008,0.057293,0.062412,0.045589,...,0.265700,0.177465,0.191235,0.277778,0.181818,0.287440,0.207715,0.252717,0.224096,0.339031


In [12]:
array1=gf.values
array2=df.values

x_feature=array1[:,0:]
x_feature1=array1[:,:]
imagelabel= x_feature[:,0]
y_label=df.label.astype('int')
print(x_feature.shape)
print(y_label.shape)

(1730, 3060)
(1730,)


In [13]:
imagelabel

array([0.20597467, 0.06967312, 0.15111907, ..., 0.05323309, 0.3290584 ,
       0.1102601 ])

In [14]:
ids=imagelabel

In [15]:
y_label

image_names
Au_ani_0008.jpg                        0
Au_ani_0017.jpg                        0
Au_ani_0027.jpg                        0
Au_ani_0010.jpg                        0
Au_ani_0003.jpg                        0
                                      ..
Sp_S_NRN_A_nat0051_nat0051_0250.jpg    2
Sp_S_NRN_R_arc0058_arc0058_0255.jpg    2
Sp_S_NRN_A_nat0098_nat0098_0003.jpg    2
Sp_S_NRN_R_arc0011_arc0045_0085.jpg    2
Sp_S_NRN_A_sec0060_sec0060_0252.jpg    2
Name: label, Length: 1730, dtype: int64

In [16]:
x_feature

array([[0.20597467, 0.19946895, 0.20630949, ..., 0.41032609, 0.29156627,
        0.43019943],
       [0.06967312, 0.04531692, 0.04635216, ..., 0.1548913 , 0.21445783,
        0.21367521],
       [0.15111907, 0.15203168, 0.1684882 , ..., 0.3423913 , 0.30361446,
        0.38461538],
       ...,
       [0.05323309, 0.05872212, 0.06517745, ..., 0.25271739, 0.22409639,
        0.33903134],
       [0.3290584 , 0.4279377 , 0.3975292 , ..., 0.22826087, 0.23373494,
        0.31339031],
       [0.1102601 , 0.11498817, 0.15150482, ..., 0.14130435, 0.26746988,
        0.17378917]])

In [17]:
y_label

image_names
Au_ani_0008.jpg                        0
Au_ani_0017.jpg                        0
Au_ani_0027.jpg                        0
Au_ani_0010.jpg                        0
Au_ani_0003.jpg                        0
                                      ..
Sp_S_NRN_A_nat0051_nat0051_0250.jpg    2
Sp_S_NRN_R_arc0058_arc0058_0255.jpg    2
Sp_S_NRN_A_nat0098_nat0098_0003.jpg    2
Sp_S_NRN_R_arc0011_arc0045_0085.jpg    2
Sp_S_NRN_A_sec0060_sec0060_0252.jpg    2
Name: label, Length: 1730, dtype: int64

In [18]:
X_train,X_test,Y_train,Y_test=train_test_split(x_feature,y_label,test_size=0.20,random_state=7)

In [19]:
X_train

array([[0.15686179, 0.18928012, 0.22594812, ..., 0.14945652, 0.26506024,
        0.13960114],
       [0.40427464, 0.37226254, 0.39939783, ..., 0.32880435, 0.3373494 ,
        0.44444444],
       [0.43914056, 0.50121948, 0.46488999, ..., 0.3451087 , 0.33493976,
        0.45299145],
       ...,
       [0.28296719, 0.32487638, 0.37854963, ..., 0.29076087, 0.37590361,
        0.45584046],
       [0.44582907, 0.45833365, 0.51741231, ..., 0.23097826, 0.35180723,
        0.21652422],
       [0.1270291 , 0.15558259, 0.12724274, ..., 0.17119565, 0.21204819,
        0.23646724]])

In [20]:
X_test

array([[0.219368  , 0.2245227 , 0.21017762, ..., 0.33967391, 0.36626506,
        0.47008547],
       [0.15533386, 0.18672084, 0.23163071, ..., 0.2826087 , 0.29879518,
        0.44159544],
       [0.11733978, 0.18355647, 0.18858691, ..., 0.36956522, 0.36385542,
        0.49287749],
       ...,
       [0.33195325, 0.33484616, 0.36118488, ..., 0.30978261, 0.32771084,
        0.4985755 ],
       [0.20201004, 0.26010122, 0.30511022, ..., 0.36684783, 0.36144578,
        0.4985755 ],
       [0.14839622, 0.17066985, 0.1715263 , ..., 0.3125    , 0.28674699,
        0.45868946]])

In [21]:
Y_test

image_names
Au_art_0014.jpg                        0
Sp_D_NRN_A_nat0032_nat0040_0529.jpg    1
Sp_D_NNN_A_cha0052_cha0052_0309.jpg    1
Sp_D_NNN_A_txt0078_txt0041_0157.jpg    1
Au_sec_0042.jpg                        0
                                      ..
Au_nat_0030.jpg                        0
Au_cha_0063.jpg                        0
Au_cha_0050.JPG                        0
Sp_S_NNN_R_sec0045_sec0045_0075.jpg    2
Sp_D_NNN_A_ani0044_ani0051_0286.jpg    1
Name: label, Length: 346, dtype: int64

In [22]:
from sklearn.svm import SVC

In [23]:
model_SVC=SVC(kernel='rbf',C=100,gamma=0.001)

kfold=KFold(n_splits=10, shuffle=True)
cv_results=cross_val_score(model_SVC,X_train,Y_train,cv=kfold,scoring='accuracy')
msg="%s %f (%f)" % ('Training Accuracy: ',cv_results.mean(),cv_results.std())
print(msg)

Training Accuracy:  0.860515 (0.031321)


In [24]:
model_SVC=SVC()

kfold=KFold(n_splits=10)
param_grid = {'C': [ 10, 100, 500, 1000], 
              'gamma': [ 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 

grid=GridSearchCV(estimator=model_SVC,param_grid=param_grid,scoring='accuracy',cv=kfold,verbose=3)
grid_result=grid.fit(X_train,Y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV 1/10] END ......C=10, gamma=0.1, kernel=rbf;, score=0.820 total time=   2.8s
[CV 2/10] END ......C=10, gamma=0.1, kernel=rbf;, score=0.777 total time=   3.1s
[CV 3/10] END ......C=10, gamma=0.1, kernel=rbf;, score=0.770 total time=   2.9s
[CV 4/10] END ......C=10, gamma=0.1, kernel=rbf;, score=0.827 total time=   2.8s
[CV 5/10] END ......C=10, gamma=0.1, kernel=rbf;, score=0.819 total time=   2.8s
[CV 6/10] END ......C=10, gamma=0.1, kernel=rbf;, score=0.812 total time=   2.9s
[CV 7/10] END ......C=10, gamma=0.1, kernel=rbf;, score=0.775 total time=   3.1s
[CV 8/10] END ......C=10, gamma=0.1, kernel=rbf;, score=0.804 total time=   2.8s
[CV 9/10] END ......C=10, gamma=0.1, kernel=rbf;, score=0.804 total time=   2.8s
[CV 10/10] END .....C=10, gamma=0.1, kernel=rbf;, score=0.754 total time=   2.8s
[CV 1/10] END .....C=10, gamma=0.01, kernel=rbf;, score=0.899 total time=   1.6s
[CV 2/10] END .....C=10, gamma=0.01, kernel=rb

In [25]:
from joblib import dump
model_SVC = SVC(C=100,gamma=0.01, kernel='rbf')
model_SVC.fit(X_train,Y_train) 
dump(model_SVC, 'svm_model.sav')

predictions=model_SVC.predict(X_test)

print(accuracy_score(Y_test,predictions))
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))

0.846820809248555
[[154   5   3]
 [ 11  60  15]
 [  3  16  79]]
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       162
           1       0.74      0.70      0.72        86
           2       0.81      0.81      0.81        98

    accuracy                           0.85       346
   macro avg       0.82      0.82      0.82       346
weighted avg       0.84      0.85      0.85       346



In [26]:
print("Accuracy:", accuracy_score(Y_test,predictions))

Accuracy: 0.846820809248555


In [27]:
predictions

array([0, 2, 0, 1, 2, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 2, 2, 0, 2, 0, 1, 0,
       1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 2,
       0, 0, 0, 1, 2, 2, 2, 1, 0, 2, 0, 1, 0, 1, 2, 2, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 2, 1, 1, 2, 1, 2, 0, 1, 2,
       0, 2, 0, 0, 0, 0, 0, 2, 1, 0, 0, 2, 1, 2, 1, 0, 0, 0, 0, 2, 0, 1,
       1, 2, 0, 0, 1, 2, 0, 2, 0, 0, 2, 1, 0, 1, 0, 0, 1, 0, 0, 0, 2, 2,
       0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 0, 0,
       0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 1, 0, 2, 1, 2, 2, 1, 0, 0, 2, 1, 0,
       0, 2, 1, 1, 0, 0, 1, 0, 0, 2, 0, 0, 2, 1, 2, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 2, 2, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0, 1, 0, 1, 2,
       1, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 1, 2, 0, 2, 1, 0, 0, 0, 2, 0, 0,
       2, 2, 2, 1, 0, 2, 2, 1, 0, 1, 0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 2,
       0, 0, 0, 0, 2, 2, 0, 2, 1, 0, 0, 0, 1, 0, 0, 1, 2, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 2, 0, 2, 0, 2, 0, 0, 2, 1, 0, 0,

In [28]:
Imagenames=Y_test.index

In [29]:
for i in range(len(Y_test)):
    if predictions[i] == Y_test[i]:
        print("CORRECT :  ", Imagenames[i], "-->Correctly classified as :", predictions[i])
    else:
        print("INCORRECT: ", Imagenames[i], "-->Incorrectly classified as :", predictions[i])

CORRECT :   Au_art_0014.jpg -->Correctly classified as : 0
INCORRECT:  Sp_D_NRN_A_nat0032_nat0040_0529.jpg -->Incorrectly classified as : 2
INCORRECT:  Sp_D_NNN_A_cha0052_cha0052_0309.jpg -->Incorrectly classified as : 0
CORRECT :   Sp_D_NNN_A_txt0078_txt0041_0157.jpg -->Correctly classified as : 1
INCORRECT:  Au_sec_0042.jpg -->Incorrectly classified as : 2
INCORRECT:  Sp_S_NNN_R_ani0094_ani0094_0049.jpg -->Incorrectly classified as : 0
CORRECT :   Sp_D_NRN_A_ani0013_ani0026_0409.jpg -->Correctly classified as : 1
CORRECT :   Sp_D_NRN_A_art0098_arc0002_0468.jpg -->Correctly classified as : 1
CORRECT :   Au_cha_0009.jpg -->Correctly classified as : 0
CORRECT :   Au_cha_0075.jpg -->Correctly classified as : 0
CORRECT :   Sp_D_NRN_A_arc0025_arc0033_0444.jpg -->Correctly classified as : 1
CORRECT :   Au_art_0025.jpg -->Correctly classified as : 0
CORRECT :   Sp_D_NRN_A_ani0080_sec0005_0426.jpg -->Correctly classified as : 1
CORRECT :   Au_sec_0089.jpg -->Correctly classified as : 0
CORREC

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
clf = RandomForestClassifier(n_estimators = 750) 
 
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(X_train, Y_train)
 
# performing predictions on the test dataset
y_pred = clf.predict(X_test)
 
# metrics are used to find accuracy or error
from sklearn import metrics 
print()
 
# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(Y_test, y_pred))


ACCURACY OF THE MODEL:  0.8063583815028902


In [33]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      0.93      0.85       162
           1       0.78      0.67      0.72        86
           2       0.88      0.71      0.79        98

    accuracy                           0.81       346
   macro avg       0.82      0.77      0.79       346
weighted avg       0.81      0.81      0.80       346



In [34]:
from sklearn.neighbors import KNeighborsClassifier

# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors = 3)

# Fit the classifier to the data
knn.fit(X_train,Y_train)

KNeighborsClassifier(n_neighbors=3)

In [35]:
knn.predict(X_test)

array([0, 0, 0, 1, 0, 2, 0, 0, 2, 0, 1, 0, 1, 0, 0, 2, 2, 0, 2, 0, 1, 0,
       0, 0, 0, 0, 2, 1, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2,
       0, 0, 0, 0, 2, 2, 0, 1, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0,
       0, 1, 0, 2, 0, 2, 1, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0, 1, 2, 2, 1, 0,
       0, 2, 1, 0, 0, 0, 1, 2, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 2, 2, 1,
       1, 0, 0, 0, 1, 0, 2, 2, 0, 2, 2, 1, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0,
       2, 0, 2, 0, 0, 2, 2, 0, 1, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 1, 0, 0,
       0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 2, 1, 1, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 2, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 0, 1, 2,
       1, 2, 0, 0, 0, 2, 2, 2, 2, 1, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 0, 2,
       2, 2, 2, 1, 0, 0, 2, 0, 0, 1, 0, 0, 2, 2, 2, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 2, 2, 0, 1, 0, 0, 2, 2, 2, 0,
       1, 1, 0, 1, 2, 0, 0, 0, 2, 0, 0, 2, 1, 1, 0,

In [36]:
knn.score(X_test,Y_test)

0.6647398843930635

In [37]:
y_pred1 = knn.predict(X_test)

In [38]:
print(classification_report(Y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.64      0.79      0.71       162
           1       0.77      0.57      0.65        86
           2       0.64      0.54      0.59        98

    accuracy                           0.66       346
   macro avg       0.68      0.63      0.65       346
weighted avg       0.67      0.66      0.66       346

